<h1> Feature Engineering </h1>

This notebook is Lab4a of CPB 102, Google's course on Machine Learning using Cloud ML.

This notebook demonstrates:
<ol>
<li> Reading data from BigQuery </li>
<li> Carrying out preprocessing using the ML SDK </li>
<li> Adding feature crosses in TensorFlow </li>
</ol>

Table of Contents:
<div id="toc"></div>

In [6]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [1]:
import google.cloud.ml as ml
import tensorflow as tf
print tf.__version__
print ml.sdk_location

0.11.0rc0
gs://cloud-ml/sdk/cloudml-0.1.6-alpha.dataflow.tar.gz


<h2> Environment variables for project and bucket </h2>

Change the cell below to reflect your Project ID and bucket name. See Lab 3a for setup instructions.

In [2]:
import os
PROJECT = 'cloud-training-demos'    # CHANGE THIS
BUCKET = 'cloud-training-demos-ml'  # CHANGE THIS

os.environ['PROJECT'] = PROJECT # for bash
os.environ['BUCKET'] = BUCKET # for bash

<h2> Specifying query to pull the data </h2>

The full dataset is 1 billion rows. For experimentation, let's sample it to create 10,000 samples.
Later, we'll remove the limit and train on the full dataset.
We're also using BigQuery sampling to pull out independent training and validation samples.

Note that because the test dataset is now different, we can not really compare test statistics between this and the previous .csv methods.

In [3]:
def create_query(phase, EVERY_N):
  """
  phase: 1=train 2=valid
  """
  base_query = """
SELECT
  DAYOFWEEK(pickup_datetime)*1.0 AS dayofweek,
  HOUR(pickup_datetime)*1.0 AS hourofday,
  pickup_longitude, pickup_latitude, 
  dropoff_longitude, dropoff_latitude,
  passenger_count*1.0 AS passenger_count,
  (tolls_amount + fare_amount) as fare_amount
FROM
  [nyc-tlc:yellow.trips]
WHERE
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0 
  """

  if EVERY_N == None:
    if phase < 2:
      # training
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 < 2".format(base_query)
    else:
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 == {1}".format(base_query, phase)
  else:
      query = "{0} AND ABS(HASH(pickup_datetime)) % {1} == {2}".format(base_query, EVERY_N, phase)

  
    
  return query
    
print create_query(2, 100000)


SELECT
  DAYOFWEEK(pickup_datetime)*1.0 AS dayofweek,
  HOUR(pickup_datetime)*1.0 AS hourofday,
  pickup_longitude, pickup_latitude, 
  dropoff_longitude, dropoff_latitude,
  passenger_count*1.0 AS passenger_count,
  (tolls_amount + fare_amount) as fare_amount
FROM
  [nyc-tlc:yellow.trips]
WHERE
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0 
   AND ABS(HASH(pickup_datetime)) % 100000 == 2


Try the query above in https://bigquery.cloud.google.com/table/nyc-tlc:yellow.trips if you want to see what it does (ADD LIMIT 10 to the query!)

<h2> Preprocessing features using Cloud ML SDK </h2>

We could discretize the lat-lon columns using the SDK, but we'll defer that to TensorFlow to enable it to be a hyper-parameter if necessary.

In [4]:
import google.cloud.ml.features as features

import google.cloud.ml as ml
print ml.sdk_location

class TaxifareFeatures(object):
  csv_columns = ('dayofweek', 'hourofday', 'pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','fare_amount')
  fare_amount = features.target('fare_amount').continuous()
  pcount = features.numeric('passenger_count').scale()
  plat = features.numeric('pickup_latitude').scale()
  dlat = features.numeric('dropoff_latitude').scale()
  plon = features.numeric('pickup_longitude').scale()
  dlon = features.numeric('dropoff_longitude').scale()
  dayofweek = features.numeric('dayofweek').identity()
  hourofday = features.numeric('hourofday').identity()

gs://cloud-ml/sdk/cloudml-0.1.6-alpha.dataflow.tar.gz


<h2> Preprocessing Dataflow job from BigQuery </h2>

This code reads from BigQuery and runs the above preprocessing, saving the data on Google Cloud.

If you are running on the Cloud, you should go to the GCP Console (https://console.cloud.google.com/dataflow) to look at the status of the job. If you are running locally, you'll get a Running bar and it will take up to 5 minutes.

In [8]:
%bash
gsutil -m rm -r -f gs://$BUCKET/taxifare/taxi_preproc4a

CommandException: 1 files/objects could not be removed.


In [9]:
# imports
import apache_beam as beam
import google.cloud.ml as ml
import google.cloud.ml.dataflow.io.tfrecordio as tfrecordio
import google.cloud.ml.io as io
import os
import datetime

# Change as needed
EVERY_N = 50 * 1000 # Change this to None to preprocess full dataset

# Direct runs locally; Dataflow runs on the Cloud.
RUNNER = 'DirectPipelineRunner'
#RUNNER = 'DataflowPipelineRunner'

OUTPUT_DIR = 'gs://{0}/taxifare/taxi_preproc4a/'.format(BUCKET)
options = {
    'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
    'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
    'job_name': 'preprocess-taxifeatures' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S'),
    'project': PROJECT,
    'extra_packages': [ml.sdk_location],
    'teardown_policy': 'TEARDOWN_ALWAYS',
    'no_save_main_session': True
}
opts = beam.pipeline.PipelineOptions(flags=[], **options)
pipeline = beam.Pipeline(RUNNER, options=opts)

# defines
feature_set = TaxifareFeatures()
train_query = create_query(1, EVERY_N)
valid_query = create_query(2, EVERY_N)
train = pipeline | 'read_train' >> beam.Read(beam.io.BigQuerySource(query=train_query))
eval = pipeline | 'read_valid' >> beam.Read(beam.io.BigQuerySource(query=valid_query))

(metadata, train_features, eval_features) = ((train, eval) |
   'Preprocess' >> ml.Preprocess(feature_set))

(metadata
   | 'SaveMetadata'
   >> io.SaveMetadata(os.path.join(OUTPUT_DIR, 'metadata.yaml')))
(train_features
   | 'WriteTraining'
   >> io.SaveFeatures(os.path.join(OUTPUT_DIR, 'features_train')))
(eval_features
   | 'WriteEval'
   >> io.SaveFeatures(os.path.join(OUTPUT_DIR, 'features_eval')))

# run pipeline
pipeline.run()

In [10]:
!gsutil ls gs://$BUCKET/taxifare/taxi_preproc4a/

gs://cloud-training-demos-ml/taxifare/taxi_preproc4a/features_eval-00000-of-00001.tfrecord.gz
gs://cloud-training-demos-ml/taxifare/taxi_preproc4a/features_train-00000-of-00001.tfrecord.gz
gs://cloud-training-demos-ml/taxifare/taxi_preproc4a/metadata.yaml


In [11]:
%bash
gsutil cp -R gs://$BUCKET/taxifare/taxi_preproc4a /content/training-data-analyst/CPB102/lab4a

Copying gs://cloud-training-demos-ml/taxifare/taxi_preproc4a/features_eval-00000-of-00001.tfrecord.gz...
Copying gs://cloud-training-demos-ml/taxifare/taxi_preproc4a/features_train-00000-of-00001.tfrecord.gz...
Copying gs://cloud-training-demos-ml/taxifare/taxi_preproc4a/metadata.yaml...
\ [3 files][  1.3 MiB/  1.3 MiB]                                                
Operation completed over 3 objects/1.3 MiB.                                      


<h2> 3. Training </h2>

Training requires you to package up your TensorFlow model into a Python package. We've done this in the directory 'taxifare'

In that code, the latitude and longitude are discretized, and feature-crossed. The hourofday and dayofweek are divided into buckets that reflect typical traffic patterns.  The whole model is then trained.

In [12]:
%bash
grep -A 40 create_inputs taxifare/trainer/taxifare.py

def create_inputs(metadata, input_data=None):
  with tf.name_scope('inputs'):
    if input_data is None:
      input_data = tf.placeholder(tf.string, name='input', shape=(None,))
    parsed = features.FeatureMetadata.parse_features(metadata, input_data)

    # [batchsize, 1] 
    plat = parsed['plat']
    plon = parsed['plon']
    dlat = parsed['dlat']
    dlon = parsed['dlon']
    dayofweek = parsed['dayofweek']
    hourofday = parsed['hourofday']

    # pickup is [batchsize, nbuckets**2]
    pickup_index, pickup = feature_cross_latlon(plat, plon, 'pickup')
    dropoff_index, dropoff = feature_cross_latlon(dlat, dlon, 'dropoff')
    # pickupdropoff is [batchsize, nbuckets**4]
    pickupdropoff = tf.squeeze(tf.one_hot(tf.mul(pickup_index, dropoff_index), depth=NUMBUCKETS**4, axis=-1), [1])
    latdist = tf.abs(tf.sub(plat, dlat))
    londist = tf.abs(tf.sub(plon, dlon))

    _print_shape(plat, 'plat')
    _print_shape(latdist, 'latdist')
    _print_shape(pickup, 'pickup')
    _print_sh

In [13]:
%bash
rm -rf taxifare.tar.gz taxi_trained
tar cvfz taxifare.tar.gz taxifare

taxifare/
taxifare/PKG-INFO
taxifare/setup.cfg
taxifare/setup.py
taxifare/trainer/
taxifare/trainer/__init__.py
taxifare/trainer/task.py
taxifare/trainer/taxifare.py
taxifare/trainer.egg-info/
taxifare/trainer.egg-info/dependency_links.txt
taxifare/trainer.egg-info/PKG-INFO
taxifare/trainer.egg-info/SOURCES.txt
taxifare/trainer.egg-info/top_level.txt


In [14]:
%mlalpha train
package_uris: /content/training-data-analyst/CPB102/lab4a/taxifare.tar.gz
python_module: trainer.task
scale_tier: BASIC
region: us-central1
args:
  train_data_paths: /content/training-data-analyst/CPB102/lab4a/taxi_preproc4a/features_train*
  eval_data_paths: /content/training-data-analyst/CPB102/lab4a/taxi_preproc4a/features_eval*
  metadata_path: /content/training-data-analyst/CPB102/lab4a/taxi_preproc4a/metadata.yaml
  output_path: /content/training-data-analyst/CPB102/lab4a/taxi_trained
  max_steps: 2500

Job Running...

master log

master: INFO:root:Step 2500: loss = 10.69 (0.008 sec) master: INFO:root:Final rmse after 2500 steps = 9.665 master: master: INFO:root:Done training. master: plat = [128, 1] master: latdist = [128, 1] master: pickup = [128, 25] master: pickupdropoff = [128, 625] master: inputs= [128, 685] master: plat = [128, 1] master: latdist = [128, 1] master: pickup = [128, 25] master: pickupdropoff = [128, 625] master: inputs= [128, 685] master: plat = [None, 1] master: latdist = [None, 1] master: pickup = [None, 25] master: pickupdropoff = [None, 625] master: inputs= [None, 685] master:

Job Finished.

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License